In [1]:
# 최신순으로 최근 90일 리뷰 크롤링
# 날짜, 리뷰 내용
# 크롬 108.xx 버전

In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException

import time
from datetime import datetime, timedelta

In [2]:
def findCafe(phone, name):
    url = f'https://m.map.naver.com/search2/searchMore.naver?query={phone}&page=1'
    response = requests.get(url)
    places = response.json()['result']['site']['list']
    for place in places:
        if place['name'] == name:
            return place['id'][1:]

In [3]:
def dateCal(str):
    date = str.split(" ")
    year = int(date[0][:-1])
    month = int(date[1][:-1])
    day = int(date[2][:-1])
    
    return datetime(year, month, day)

In [5]:
cafeNames = [
    '빵굽는커피공장',
    '글루미',
    '마마다이닝'
]

cafeNums = [
    '053-422-3833', # 빵굽는커피공장
    '0507-1331-5449', # 글루미
    '0507-1374-2778' # 마마다이닝
]

cafeIDs = []

In [6]:
for cafeName, cafeNum in zip(cafeNames, cafeNums):
    cafeID = findCafe(cafeNum, cafeName)
    cafeIDs.append(cafeID)
    time.sleep(5)

In [7]:
cafeIDs

['19675007', '1431782782', '1696700298']

In [8]:
def clickMore(driver, days=90):
    now = datetime.now()

    while True: 
            try: 
                driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
                time.sleep(1)
                driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_UP)
                time.sleep(1) 
                
                date = driver.find_element(By.CSS_SELECTOR, '.eCPGL > li:nth-last-child(1) > div:nth-last-child(1) > span > span:nth-last-child(1)').text
                
                dt = dateCal(date)
                diff = (now-dt).total_seconds()//86400
                
                if diff > days: # 맨 마지막 리뷰 일자가 90일 이전이면 멈춤
                    break
                
                driver.find_element(By.CSS_SELECTOR, '.lfH3O > a').click()
                time.sleep(2)           
                
                
            except NoSuchElementException: 
                print('-더보기 버튼 모두 클릭 완료-') 
                break 

In [9]:
def seeDetails(driver, action):
    elements = driver.find_elements(By.CSS_SELECTOR, '.YeINN')

    for element in elements:
        more = element.find_elements(By.CSS_SELECTOR, '.ZZ4OK > a > .rvCSr')
        if len(more) > 0:
            action.move_to_element(element).perform()
            time.sleep(1)
            more[0].click()

In [10]:
def scrapReviews(html, cafename, cafelist, datelist, reviewlist):
    dom = BeautifulSoup(html, 'lxml')

    reviews = dom.select('.YeINN > .ZZ4OK > a > span:nth-child(1)')
    dates = dom.select('.YeINN > div:nth-last-child(1) > span > span:nth-last-child(1)')

    try: 
        for date, review in zip(dates, reviews): 
            
            day = dateCal(date.text)
            text = review.text
        
            cafelist.append(cafename)
            datelist.append(day)
            reviewlist.append(text)

    # 리뷰가 없는 경우        
    except NoSuchElementException: 
        print("네이버 리뷰 없음" )

In [11]:
# options = webdriver.ChromeOptions()
# options.add_argument('headless')
# driver = webdriver.Chrome(options=options)

driver = webdriver.Chrome()
action = ActionChains(driver)

In [12]:
cafes = []
days = []
re = []

for cafeID, cafeName in zip(cafeIDs, cafeNames):
    finalurl = f'https://m.place.naver.com/restaurant/{cafeID}/review/visitor?reviewSort=recent'
    driver.get(finalurl)

    clickMore(driver)
    time.sleep(2)

    seeDetails(driver, action)

    html = driver.page_source

    scrapReviews(html, cafeName, cafes, days, re)
    print(f'{cafeName} 리뷰 수집 완료')
    time.sleep(3)

driver.quit()

빵굽는커피공장 리뷰 수집 완료
글루미 리뷰 수집 완료
마마다이닝 리뷰 수집 완료


In [13]:
df = pd.DataFrame({'cafeName':cafes,
                   'date': days,
                   'review': re
                   })

In [14]:
df.head()

,cafeName,date,review
0,빵굽는커피공장,2022-12-17,자리가 편안하고 빵도 같이 주셔서 좋아요
1,빵굽는커피공장,2022-12-17,대구 시내에 이렇게 아늑한 분위기의 커피숍이 있는지 몰랐어요..옛날 다방같은 느낌입...
2,빵굽는커피공장,2022-12-11,커피가 맛있고 분위기가 좋아요
3,빵굽는커피공장,2022-12-02,조용하고 분위기 넘 좋아요
4,빵굽는커피공장,2022-11-19,조아용


In [15]:
df.tail()

,cafeName,date,review
366,마마다이닝,2022-10-01,푸딩빙수가 예쁜데 맛도 있어요
367,마마다이닝,2022-10-01,구슬아이스크림 빙수인데 위에 올려주신 구슬아이스크림에서 아무맛이 안나요.. 그냥 맹...
368,마마다이닝,2022-10-01,파리나방 계속 날아다님 ㅜㅜ. 다시는 안갈집
369,마마다이닝,2022-10-01,👍
370,마마다이닝,2022-09-30,ㅎ


In [16]:
now = datetime.now()
tmp = now + timedelta(days=-90)

In [17]:
condition = df.loc[df['date']>=tmp]

In [18]:
condition.to_csv('./data/test_raw.csv', index=False, encoding='utf-8-sig')